In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
# artigo_treino = pd.read_csv("../../../global-assets/")
# artigo_teste = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/teste.csv")

df = pd.read_csv("../../../global-assets/datasets/amazon-reviews.csv")

artigo_treino, artigo_teste = train_test_split(df, test_size=0.2, random_state=42)


In [10]:
artigo_treino.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
251349,251350,B004H3N2LU,A3T60XAQ0WUTB0,"astro128 ""astro128""",3,3,1,1299888000,Terrible Product,"Like many others have said, this is INSTANT co..."
523262,523263,B000FPX7SW,AZAZ4IO7QRQE7,"K. Meyer ""kat lady""",6,7,5,1170720000,Delicious!,We are a 2 person household with one diabetic ...
224543,224544,B000EQT574,AZOYVQTM4DWAX,"techrockstar ""the techrockstar""",0,0,5,1294185600,Delicious!,These might be the best chips I have ever had....
291632,291633,B004H1WJEI,AFOEOYSYD04J5,Gerald M Crowley,1,1,4,1322611200,contains Fo Ti,which is why I purchased it for a snack. But t...
37385,37386,B000LQNC30,A1ZBLEHXI5UZLL,Saleena Shamin,0,0,5,1189555200,Great,Everything was perfect except the fact that th...


In [8]:
artigo_teste.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
165256,165257,B000EVG8J2,A1L01D2BD3RKVO,"B. Miller ""pet person""",0,0,5,1268179200,Crunchy & Good Gluten-Free Sandwich Cookies!,Having tried a couple of other brands of glute...
231465,231466,B0000BXJIS,A3U62RE5XZDP0G,Marty,0,0,5,1298937600,great kitty treats,My cat loves these treats. If ever I can't fin...
427827,427828,B008FHUFAU,AOXC0JQQZGGB6,Kenneth Shevlin,0,2,3,1224028800,COFFEE TASTE,A little less than I expected. It tends to ha...
433954,433955,B006BXV14E,A3PWPNZVMNX3PA,rareoopdvds,0,1,2,1335312000,So the Mini-Wheats were too big?,"First there was Frosted Mini-Wheats, in origin..."
70260,70261,B007I7Z3Z0,A1XNZ7PCE45KK7,Og8ys1,0,2,5,1334707200,Great Taste . . .,and I want to congratulate the graphic artist ...


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

texto = [
    "Este produto é muito bom",
    "Este produto é muito ruim"
]

vetorizador = CountVectorizer()
vetorizador.fit(texto)
print(vetorizador.vocabulary_)

{'este': 1, 'produto': 3, 'muito': 2, 'bom': 0, 'ruim': 4}


In [13]:
vetor_bom = vetorizador.transform(["bom"])
print(vetor_bom.toarray())

[[1 0 0 0 0]]


In [18]:
from gensim.models import KeyedVectors
modelo = KeyedVectors.load_word2vec_format("../../../global-assets/datasets/cbow_s300.txt")


In [19]:
modelo.most_similar(positive=["carros", "foguetes"], negative=["carro"])

[('foguetões', 0.5806891918182373),
 ('aviões', 0.5474070906639099),
 ('caças', 0.5254919528961182),
 ('icbms', 0.521876871585846),
 ('satélites', 0.5179443359375),
 ('projéteis', 0.5173476934432983),
 ('motores', 0.516154944896698),
 ('vaivéns', 0.5121830701828003),
 ('mísseis', 0.5111415386199951),
 ('obuses', 0.5107772946357727)]

In [20]:
import nltk
import string
nltk.download('punkt')

def tokenizador(texto):
  texto = texto.lower()
  lista_alfanumerico = []

  for token_valido in nltk.word_tokenize(texto):
    if token_valido in string.punctuation: continue
    lista_alfanumerico.append(token_valido)
  
  return lista_alfanumerico

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kake\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
tokenizador("Texto com pontuação")

['texto', 'com', 'pontuação']

In [22]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras_numeros):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try:
      vetor_resultante =+ modelo.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = "0"*len(pn)
        vetor_resultante =+ modelo.get_vector(pn)
      else:
        vetor_resultante =+ modelo.get_vector('unknown')
  return vetor_resultante

In [23]:
def matriz_vetores(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)

  return matriz

In [26]:
matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)

AttributeError: 'DataFrame' object has no attribute 'title'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LR = LogisticRegression(max_iter = 200)
LR.fit(matriz_vetores_treino, artigo_treino.category)
label_prevista = LR.predict(matriz_vetores_teste)
CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

In [ ]:
from sklearn.dummy import DummyClassifier

DC = DummyClassifier()
DC.fit(matriz_vetores_treino, artigo_treino.category)
label_previsao_dc = DC.predict(matriz_vetores_teste)
CR_dummy = classification_report(artigo_teste.category, label_previsao_dc)
print(CR_dummy)

In [ ]:
modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

In [ ]:
def combinacao_de_vetores_por_soma_skipgram(palavras_numeros):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try:
      vetor_resultante =+ modelo_skipgram.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = "0"*len(pn)
        vetor_resultante =+ modelo_skipgram.get_vector(pn)
      else:
        vetor_resultante =+ modelo_skipgram.get_vector('unknown')
  return vetor_resultante

In [ ]:
def matriz_vetores_skipgram(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma_skipgram(palavras_numeros)

In [ ]:
matriz_vetores_treino_skipgram = matriz_vetores_skipgram(artigo_treino.title)
matriz_vetores_teste_skipgram = matriz_vetores_skipgram(artigo_teste.title)

In [ ]:
LR_skipgram = LogisticRegression(max_iter = 200)
LR_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)
label_previsao_skipgram = LR_skipgram.predict(matriz_vetores_teste_skipgram)
CR_skipgram = classification_report(artigo_teste.category, label_previsao_skipgram)
print(CR_skipgram)